In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import tqdm as tqdm
from collections import defaultdict
import pickle as pickle

def get_tfidf(column):
	tfidf_vectorizer = TfidfVectorizer(stop_words ="english")
	tfidf_matrix = tfidf_vectorizer.fit_transform(column.values.astype(str))
	return tfidf_matrix

def find_similar(tfidf_matrix, index, top_n = 5):
    cosine_similarities = linear_kernel(tfidf_matrix[index:index+1], tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)	

answers_file = "CS_answers.csv"
df = pd.read_csv(answers_file)
saved_column = df.Body #you can also use df['column_name']
saved_column.fillna(" ")
tfidf_answers = get_tfidf(saved_column)

answers_rec_dict = defaultdict(list)
for i in tqdm.tqdm(range(saved_column.shape[0])):
    for index, score in find_similar(tfidf_answers, i):
        if df['Id'][i] in answers_rec_dict:
            (answers_rec_dict[df['Id'][i]]).append((df['Id'][index],score))
        else:
            answers_rec_dict[df['Id'][i]] = [(df['Id'][index],score)]




100%|██████████| 35056/35056 [44:09<00:00, 13.23it/s]


NameError: ignored

In [11]:
save_obj(answers_rec_dict,"tf_idf_answers_rec")
			

questions_file = "CS_questions.csv"
df_ques = pd.read_csv(questions_file)
df_question_merged = df_ques.Body.astype(str) + " " + df_ques.Title.astype(str) + " " + df_ques.Tags.astype(str)
tfidf_questions_merged = get_tfidf(df_question_merged)

questions_rec_dict = defaultdict(list)
for i in tqdm.tqdm(range(tfidf_questions_merged.shape[0])):
    for index, score in find_similar(tfidf_questions_merged, i):
        if df_ques['Id'][i] in questions_rec_dict:
            (questions_rec_dict[df_ques['Id'][i]]).append((df_ques['Id'][index],score))
        else:
            questions_rec_dict[df_ques['Id'][i]] = [(df_ques['Id'][index],score)]

save_obj(questions_rec_dict,"tf_idf_ques_rec")

100%|██████████| 29375/29375 [29:36<00:00, 16.53it/s]


In [5]:
import pandas as pd
postlinksDF = pd.read_csv('../csv_files/postlinks.csv')

In [6]:
postlinksDF.head()

,Id,PostId,RelatedPostId,LinkTypeId
0,865,155,143,1
1,891,143,155,1
2,1139,192,57,1
3,1789,245,256,1
4,2699,341,302,1


In [7]:
import tqdm 
#final list of dict to keep track of related post ids and answer ids for each question id
postLinks = []
#to keep track of questions that were already encountered
posts = [] 

for index, row in tqdm.tqdm(postlinksDF.iterrows()):
    temp = {'postID': None,'relatedID': [],'answerID': [] } 
    temp['postID'] = row['PostId']
    if(row['PostId'] in posts and len(posts)!=0):
        for i in range(0,len(posts)):
            if(row['PostId'] == postLinks[i]['postID']):
                postLinks[i]['relatedID'].append(row["RelatedPostId"])
                postLinks[i]['answerID'].append(row["Id"])
    else:  
        temp['relatedID'].append(row["RelatedPostId"])
        temp['answerID'].append(row["Id"])   
        posts.append(temp['postID'])
        postLinks.append(temp)

9855it [02:52, 57.09it/s] 


In [8]:
len(postLinks)

6579

In [9]:
import pickle as pickle

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

tf_idf_ques_rec = load_obj("tf_idf_ques_rec")
recommended = tf_idf_ques_rec

In [10]:
recommended[7]

[(85662, 0.3608988407573669),
 (51954, 0.2890118160092387),
 (45188, 0.2635945608870541),
 (32703, 0.2496637310035562),
 (71993, 0.24118704749672484)]

In [11]:
def evaluate():
    score  = 0
    for i in range(0,len(postLinks)):
        present = 0 
        top = 0
        count = len(postLinks[i]['relatedID'])
        if postLinks[i]['postID'] in recommended.keys():
            idx = 0
            for rec in recommended[postLinks[i]['postID']]:
                idx+=1
                if rec[0] in postLinks[i]['relatedID']:
                    present+=1
                    if (idx >= int(len(postLinks[i]['relatedID'])/2)):
                        top+=1
            if(present):
                score+= (0.1* (present/count)+ 0.9*(top/present))
    return (score/len(postLinks))

In [12]:
evaluate()*100

14.356423330431559

In [13]:
tf_idf_answers_rec = load_obj("tf_idf_answers_rec")
recommended = tf_idf_answers_rec

In [14]:
evaluate()*100

0.0